# 🏭 Training Indoor ResNet-50 Models on Real and Mixed Fire Datasets

This notebook trains ResNet-50 classification models for **indoor fire detection** using two dataset compositions:
- 100% real images from the PLOS ONE indoor dataset
- 50/50 mixed composition using the PLOS ONE real dataset combined with SYN-FIRE synthetic positives

The goal is to evaluate best-case indoor performance for deployment scenarios (e.g., factories, enclosed environments) and to assess whether supplementing limited real data with synthetic positives improves classification.

Training is performed using ResNet-50 in frozen feature extraction mode for comparability with Phase 1 results. Models are validated on the held-out PLOS ONE validation set, and final performance is measured on the PLOS ONE test set. Results will be compared directly to outdoor-trained models from Phases 1–3 to demonstrate the impact of indoor-focused training.


## 📦 Notebook Setup: Mount Drive & Clone GitHub Repo

This cell ensures the notebook is reproducible in any new Colab session by:

- Mounting your Google Drive (to access datasets, secrets, and checkpoints)
- Loading your GitHub token from Drive
- Cloning the fire-detection-dissertation repository
- Navigating into the correct folder
- Setting Git identity for future commits

⚠️ **Note:** This cell must be run every time you open this notebook in a new Colab session.


In [17]:
# 🔧 Minimal Colab setup for any working notebook

# 1. Mount Google Drive
import os
from google.colab import drive
if not os.path.ismount("/content/drive"):
    drive.mount("/content/drive")

# 2. Load GitHub token securely from Drive
token_path = "/content/drive/MyDrive/fire-detection-dissertation/secrets/github_token.txt"
with open(token_path, "r") as f:
    token = f.read().strip()

# 3. Clone the GitHub repo (force fresh clone for safety)
username = "Misharasapu"
repo = "fire-detection-dissertation"
clone_url = f"https://{token}@github.com/{username}/{repo}.git"
repo_path = f"/content/{repo}"

# Optional: Remove old clone (safe to rerun)
!rm -rf {repo_path}

# Clone fresh and move into the repo
%cd /content
!git clone {clone_url}
%cd {repo}

# 4. Set Git identity (required in Colab sessions)
!git config --global user.name "Misharasapu"
!git config --global user.email "misharasapu@gmail.com"


/content
Cloning into 'fire-detection-dissertation'...
remote: Enumerating objects: 140, done.
remote: Counting objects: 100% (140/140), done.
remote: Compressing objects: 100% (103/103), done.
remote: Total 140 (delta 69), reused 102 (delta 35), pack-reused 0 (from 0)
Receiving objects: 100% (140/140), 4.33 MiB | 32.38 MiB/s, done.
Resolving deltas: 100% (69/69), done.
/content/fire-detection-dissertation


## 🧩 Step 1: Define Dataset Type, Paths, and Model Filenames

In this step, we set up the **Phase 4 indoor training configuration**. Only **one model is trained at a time**, controlled by the `selected_model` variable:

- 🟢 **indoor_real**: 100% real images from the **PLOS ONE** dataset (indoor)  
- 🟠 **indoor_mixed**: 50% PLOS ONE real + 50% SYN-FIRE synthetic positives  
  > **Note:** The SYN-FIRE dataset contains **only positive (fire) images**, stored under  
  `/synthetic/syn-fire/synthetic_all/images` and `/synthetic/syn-fire/synthetic_all/masks`.  
  It is used **only for training augmentation**. Validation and test remain **PLOS ONE only**.

We also define model filenames using the unified naming convention (`resnet_<domain>_<composition>_phase4.pt`) and set a fixed random seed for reproducibility.  
An optional `total_samples` cap can be applied to mirror Phase 1 fairness (e.g., 5,260 samples). Leaving it as `None` will use the full PLOS ONE training split.


In [36]:
# --- Step 1: Dataset types, paths, filenames (Phase 4: Indoor) ---

import os
import random
import torch
import numpy as np


# ✅ Paths to training/val/test data (Phase 4: indoor focus)
plos_train_img_dir  = "/content/drive/MyDrive/fire-detection-dissertation/data/raw/real/PLOS_ONE/train/images"
plos_train_lbl_dir  = "/content/drive/MyDrive/fire-detection-dissertation/data/raw/real/PLOS_ONE/train/labels"
plos_val_img_dir    = "/content/drive/MyDrive/fire-detection-dissertation/data/raw/real/PLOS_ONE/valid/images"
plos_val_lbl_dir    = "/content/drive/MyDrive/fire-detection-dissertation/data/raw/real/PLOS_ONE/valid/labels"
plos_test_img_dir   = "/content/drive/MyDrive/fire-detection-dissertation/data/raw/real/PLOS_ONE/test/images"
plos_test_lbl_dir   = "/content/drive/MyDrive/fire-detection-dissertation/data/raw/real/PLOS_ONE/test/labels"

# SYN-FIRE (indoor synthetic positives only, used for mixed model)
synfire_img_dir     = "/content/drive/MyDrive/fire-detection-dissertation/data/raw/synthetic/syn-fire/synthetic_all/images"
synfire_mask_dir    = "/content/drive/MyDrive/fire-detection-dissertation/data/raw/synthetic/syn-fire/synthetic_all/masks"

# ✅ Model filenames (consistent with naming convention)
model_paths = {
    "indoor_real":   "resnet_indoor_real_100_phase4.pt",
    "indoor_mixed":  "resnet_indoor_50syn_50real_phase4.pt",
}

# ✅ Select which model to train for this run
# Options: "indoor_real" or "indoor_mixed"
selected_model = "indoor_mixed"

# ✅ Reproducibility
seed = 42
random.seed(seed)
np.random.seed(seed)
torch.manual_seed(seed)
if torch.cuda.is_available():
    torch.cuda.manual_seed_all(seed)

print(f"Phase 4 setup complete. Selected model: {selected_model}")
print("Checkpoint will be saved as:", model_paths[selected_model])
print("PLOS train path:", plos_train_img_dir)
if selected_model == "indoor_mixed":
    print("SYN-FIRE positives path:", synfire_img_dir)


Phase 4 setup complete. Selected model: indoor_mixed
Checkpoint will be saved as: resnet_indoor_50syn_50real_phase4.pt
PLOS train path: /content/drive/MyDrive/fire-detection-dissertation/data/raw/real/PLOS_ONE/train/images
SYN-FIRE positives path: /content/drive/MyDrive/fire-detection-dissertation/data/raw/synthetic/syn-fire/synthetic_all/images


## 💠 Step 2: Define Image Transformations

We define the basic preprocessing pipeline used for both training and validation.  
Images are resized to 224×224 and converted to tensors.  

🚫 No normalization is applied — all models from Phase 1 to Phase 4 are now trained and evaluated on **unnormalised images** for consistency.  
This ensures that training, validation, and testing all use the same input format and allows easier visual inspection of images.  

No data augmentation is applied.  
The test set will be handled separately in the evaluation notebook.


In [37]:
# --- Step 2: Image transforms (train + val only, no augmentation, no Normalize) ---

from torchvision import transforms

transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),            # 👈 no Normalize
])

print("Transforms set: Resize(224,224) → ToTensor()  [no normalization]")


Transforms set: Resize(224,224) → ToTensor()  [no normalization]


## ⚙️ Step 3: Load dataset and build DataLoaders

We import the dataset helper from the repo and load the **PLOS ONE (indoor)** splits.  
Only train and valid are used in this notebook. Test is handled in the evaluation notebook.

- Helper: `FireClassificationDataset` with `dataset_type="plos"` so class 0 → fire
- Splits: use the existing `train/` and `valid/` folders
- Modularity: one model per run. Currently `selected_model = "indoor_real"`. Switching later to `"indoor_mixed"` will change only the train loader.


In [38]:
# --- Step 3: Load dataset and build DataLoaders (train + valid only) ---

import importlib
import utils.fire_classification_dataset as fcd
importlib.reload(fcd)  # 🔄 ensures notebook sees latest version

from torch.utils.data import DataLoader


if selected_model == "indoor_real":
    # PLOS-only (train + valid)
    train_dataset = fcd.FireClassificationDataset(
        image_dir=plos_train_img_dir,
        label_dir=plos_train_lbl_dir,
        transform=transform,
        dataset_type="plos",
    )
else:  # indoor_mixed
    # Fixed 50/50: 2000 PLOS real + 2000 SYN-FIRE (train only)
    train_dataset = fcd.FireClassificationPhase4MixedFixed(
        real_image_dir=plos_train_img_dir,
        real_label_dir=plos_train_lbl_dir,
        syn_image_dir=synfire_img_dir,
        syn_mask_dir=synfire_mask_dir,
        n_real=2000,
        n_syn=2000,
        transform=transform,
        seed=42,
    )

# Validation stays PLOS-only
val_dataset = fcd.FireClassificationDataset(
    image_dir=plos_val_img_dir,
    label_dir=plos_val_lbl_dir,
    transform=transform,
    dataset_type="plos",
)

batch_size = 32
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True, num_workers=2)
val_loader   = DataLoader(val_dataset,   batch_size=batch_size, shuffle=False, num_workers=2)

print(f"Selected model: {selected_model}")
print(f"Train samples: {len(train_dataset)} | Valid samples: {len(val_dataset)}")
print(f"Batch size: {batch_size}")


Selected model: indoor_mixed
Train samples: 4000 | Valid samples: 500
Batch size: 32


In [39]:
# --- Debug: Confirm composition of mixed dataset ---
if selected_model == "indoor_mixed":
    # Count samples
    n_total = len(train_dataset)
    n_real  = len(train_dataset.real_indices)
    n_syn   = len(train_dataset.syn_indices)

    print(f"📊 Indoor mixed dataset composition → Real: {n_real}, Synthetic: {n_syn}, Total: {n_total}")

    # Sanity check: label distribution
    labels = [int(train_dataset[i][1]) for i in range(200)]  # sample first 200 for speed
    print(f"✅ Sample label distribution (first 200): Fire={sum(labels)}, No Fire={200 - sum(labels)}")


📊 Indoor mixed dataset composition → Real: 2000, Synthetic: 2000, Total: 4000
✅ Sample label distribution (first 200): Fire=152, No Fire=48


## 🧰 Step 4: Build ResNet-50 (feature extraction), define loss & optimizer

We follow the exact **Phase-1 style**: load a pretrained ResNet-50, **freeze the backbone**, replace the final `fc` with a **2-class** head, and train **only** that head.  
Use `CrossEntropyLoss` (logits → 2 classes) and `Adam` on the head parameters. The checkpoint file name comes from `model_paths[selected_model]`.



In [40]:
# --- Step 4: Model (feature extraction), loss, optimizer ---

import os
import torch
import torch.nn as nn
from torchvision import models

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# 1) Pretrained ResNet-50 backbone
model = models.resnet50(pretrained=True)

# 2) Freeze all backbone layers (feature extraction mode)
for p in model.parameters():
    p.requires_grad = False

# 3) Replace classification head with 2-class layer
in_features = model.fc.in_features  # typically 2048
model.fc = nn.Linear(in_features, 2)

# 4) Unfreeze the new head explicitly
for p in model.fc.parameters():
    p.requires_grad = True

# 5) Move to device
model = model.to(device)

# 6) Loss & optimizer (only head will be trained)
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.fc.parameters(), lr=1e-4)

# 7) Checkpoint path
models_dir = "/content/drive/MyDrive/fire-detection-dissertation/models"
os.makedirs(models_dir, exist_ok=True)
checkpoint_path = os.path.join(models_dir, model_paths[selected_model])

# 8) Sanity check
trainable = sum(p.numel() for p in model.parameters() if p.requires_grad)
total     = sum(p.numel() for p in model.parameters())
print(f"Device: {device} | Trainable params: {trainable} / {total} (only fc)")
print(f"Checkpoint will be saved to: {checkpoint_path}")


/usr/local/lib/python3.11/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet50_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet50_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


Device: cuda | Trainable params: 4098 / 23512130 (only fc)
Checkpoint will be saved to: /content/drive/MyDrive/fire-detection-dissertation/models/resnet_indoor_50syn_50real_phase4.pt


## 🏃‍♂️ Step 5: Train the model (feature extraction baseline)

We use the `train_model()` helper from `utils/train_model.py` to run a standard epoch loop:

- Uses GPU if available
- Tracks train/valid loss each epoch
- Saves the checkpoint **only when val F1 improves** to  
  `models/{model_paths[selected_model]}`

Training runs for `num_epochs` (default 5) to stay consistent with earlier notebooks.


In [ ]:
# --- Step 5: Train using the repo helper ---

from utils.train_model import train_model

# epochs (keep consistent with earlier phases)
num_epochs = 5

train_losses, val_losses = train_model(
    model=model,
    train_loader=train_loader,
    val_loader=val_loader,
    criterion=criterion,
    optimizer=optimizer,
    num_epochs=num_epochs,
    device=device,
    save_path=checkpoint_path,  # e.g., models/resnet_indoor_real_100_phase4.pt
    print_every=1,              # print one summary per epoch
    print_batch_loss=False      # set True only for debugging
)

print("\n✅ Training complete.")
print(f"📦 Best model saved to: {checkpoint_path}")



🔍 Model device: cuda:0
📌 Feature extraction mode: Notebook is responsible for freezing/unfreezing.
📊 Trainable parameters: 4098

🔁 Epoch 1/5


✅ Epoch [1/5] | Train Loss: 0.3785 | Val Loss: 0.4209 | Acc: 0.8700 | Precision: 0.8205 | Recall: 0.9931 | F1: 0.8986 | Time: 195.1s
💾 New best model saved (F1: 0.8986) → /content/drive/MyDrive/fire-detection-dissertation/models/resnet_indoor_50syn_50real_phase4.pt

🔁 Epoch 2/5


✅ Epoch [2/5] | Train Loss: 0.2057 | Val Loss: 0.3021 | Acc: 0.9120 | Precision: 0.8994 | Recall: 0.9552 | F1: 0.9264 | Time: 195.0s
💾 New best model saved (F1: 0.9264) → /content/drive/MyDrive/fire-detection-dissertation/models/resnet_indoor_50syn_50real_phase4.pt

🔁 Epoch 3/5


✅ Epoch [3/5] | Train Loss: 0.1528 | Val Loss: 0.2577 | Acc: 0.9160 | Precision: 0.8974 | Recall: 0.9655 | F1: 0.9302 | Time: 195.2s
💾 New best model saved (F1: 0.9302) → /content/drive/MyDrive/fire-detection-dissertation/models/resnet_indoor_50syn_50real_phase4.pt

🔁 Epoch 4/5


✅ Epoch [4/5] | Train Loss: 0.1301 | Val Loss: 0.2266 | Acc: 0.9180 | Precision: 0.9278 | Recall: 0.9310 | F1: 0.9294 | Time: 195.9s

🔁 Epoch 5/5


🚂 Training:  90%|████████▉ | 112/125 [02:52<00:21,  1.67s/it]

In [ ]:
# ✅ 1. Navigate to the Git-tracked repo folder
%cd /content/fire-detection-dissertation

# ✅ 2. Move the notebook from Drive into the repo so Git can track it
!mv /content/drive/MyDrive/fire-detection-dissertation/notebooks/07_train_resnet_indoor_models.ipynb /content/fire-detection-dissertation/notebooks/

# Optional: confirm it's now inside the repo
!ls notebooks/

# ✅ 3. Stage the notebook for commit
!git add notebooks/07_train_resnet_indoor_models.ipynb

# ✅ 4. Commit with a message
!git commit -m "Added Phase 4 indoor training notebook (PLOS ONE real + SYN-FIRE mixed, unnormalised pipeline)"

# ✅ 5. Push to GitHub
!git push
